In [3]:
import sys,os,os.path
sys.path.append("../../")
sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
os.environ['ICTDIR']='/home/e78368jw/Documents/NEXT_CODE/IC'

#%load_ext autoreload
#%autoreload 2
#%matplotlib notebook

import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['mathtext.fontset'] = 'stix'
rcParams['font.family'] = 'STIXGeneral'
rcParams['figure.figsize'] = [10, 8]
rcParams['font.size'] = 22

import pandas as pd
import numpy  as np
import tables as tb

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as clrs

import invisible_cities.core.core_functions                   as     coref
import invisible_cities.io.dst_io                           as     dstio

from invisible_cities.cities.beersheba                 import cut_over_Q, drop_isolated, deconvolve_signal


from invisible_cities.database.load_db       import DataSiPM

from invisible_cities.evm.event_model        import Cluster, Hit
from invisible_cities.types.ic_types         import xy
from invisible_cities.reco.paolina_functions import voxelize_hits

from invisible_cities.evm.event_model        import HitEnergy
from invisible_cities.cities.beersheba          import DeconvolutionMode
from invisible_cities.cities.beersheba          import CutType


from invisible_cities.reco.deconv_functions import deconvolve
from invisible_cities.reco.deconv_functions import deconvolution_input
from invisible_cities.reco.deconv_functions import InterpolationMethod

import invisible_cities.io.mcinfo_io as mcio


import matplotlib.cm as cm

In [4]:
import glob

# Input file and some event IDs to look at.

typefile = 'sophronia' #'esmeralda'
RUN = 15589
fpath = f'/home/e78368jw/Downloads/test_data/15589/sophronia/trigger2/ldc1/'
#fpath = f'/data/halmazan/NEXT/N100_LPR/15589/sophronia/prod/ldc1/'
#file_pattern = f'{fpath}run_{RUN}_*_ldc1_trg2.v2.3.1.20250708.HEDesman.sophronia.h5'
#file_pattern = f'{fpath}run_{RUN}_*_ldc1_trg2.v2.3.1.20250429.HEDesman.sophronia.h5'
file_pattern = f'{fpath}run_{RUN}_*'
files = glob.glob(file_pattern)

print(file_pattern)
# Select the first file that matches the pattern
file = files[0] if files else None

/home/e78368jw/Downloads/test_data/15589/sophronia/trigger2/ldc1/run_15589_*


In [5]:
hdst = dstio.load_dsts(files[:3], 'RECO', 'Events')

In [ ]:
display(hdst)
Xs = hdst.X.unique()
Ys = hdst.Y.unique()
#display(Xs)
display(np.diff(Xs)/15.55) # division by pitch
#display((np.diff(Ys)/15.55))

,event,time,npeak,Xpeak,Ypeak,nsipm,X,Y,Xrms,Yrms,Z,Q,E,Qc,Ec,track_id,Ep
0,13819,1.751991e+09,24,61.594370,91.802216,1,42.075,492.725,0.0,0.0,827.554625,7.252683,1.310307,-1.0,NaN,-1,-1.0
1,13819,1.751991e+09,24,61.594370,91.802216,1,490.025,29.725,0.0,0.0,835.408625,8.344430,8.237137,-1.0,NaN,-1,-1.0
2,13819,1.751991e+09,24,61.594370,91.802216,1,-467.075,276.525,0.0,0.0,855.555750,7.485704,2.097032,-1.0,NaN,-1,-1.0
3,13819,1.751991e+09,24,61.594370,91.802216,1,443.375,-217.075,0.0,0.0,867.871625,10.082024,24.904780,-1.0,NaN,-1,-1.0
4,13819,1.751991e+09,24,61.594370,91.802216,1,73.175,492.725,0.0,0.0,871.500000,7.117067,275.655669,-1.0,NaN,-1,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103554,24893,1.751991e+09,33,-9.146233,79.365734,1,427.825,168.675,0.0,0.0,914.807125,7.276436,37.726301,-1.0,0.000257,-1,-1.0
103555,24893,1.751991e+09,33,-9.146233,79.365734,1,73.175,-247.675,0.0,0.0,942.435375,8.450523,24.490590,-1.0,0.000129,-1,-1.0
103556,24893,1.751991e+09,33,-9.146233,79.365734,1,257.775,214.325,0.0,0.0,942.435375,7.319254,21.212040,-1.0,0.000119,-1,-1.0
103557,24893,1.751991e+09,33,-9.146233,79.365734,1,-359.225,-232.625,0.0,0.0,955.338125,7.456758,18.517143,-1.0,0.000109,-1,-1.0


array([ 28.80707395, -61.54983923,  58.54983923, -23.80707395,
        10.93569132,   1.93569132,  -1.        , -18.80707395,
        16.87138264,   3.93569132,  -4.93569132,   5.93569132,
         1.        ,   6.93569132, -56.54983923,  35.74276527,
        -6.93569132,   0.93569132,   1.        ,  -2.93569132,
       -21.87138264,  18.87138264,   1.        ,   6.93569132,
         5.93569132, -22.80707395, -12.87138264,   2.        ,
        16.80707395,  19.87138264,   8.93569132, -10.93569132,
       -27.74276527,  40.67845659, -32.74276527, -21.87138264,
        18.87138264, -14.87138264,   6.93569132,   4.        ,
        41.61414791,  -1.        , -30.74276527,  -8.93569132,
         1.        ,   3.        ,   3.        ,  -9.93569132,
        -6.93569132,  55.54983923, -49.61414791,  -8.93569132,
        39.67845659,  -3.93569132,   0.93569132,  20.87138264,
       -50.61414791,  -1.        ,  44.67845659, -32.74276527,
        15.87138264, -10.93569132,  31.74276527])

In [25]:
# load the detector DB
from invisible_cities.database.load_db import DataSiPM

sipm_data = DataSiPM('next100', 15589)

In [31]:
print(np.unique(np.diff(sipm_data.X)))

[-108.85 -108.85 -108.85 -108.85    0.     14.55   14.55   14.55   14.55
   14.55   15.55   15.55   15.55   15.55   15.55   15.55   15.55]
